### Redis Client

In [1]:
import pandas as pd
import pyarrow as pa
import redis



In [2]:
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=6379,db=0)

In [3]:
def get_redis_df(key):
    context = pa.default_serialization_context()
    df = context.deserialize(redis_db.get(key))
    return df

### Get Data

In [4]:
#df_spy,df_vix,df_tnx,df_1yr_rate,df_div
df_spy = get_redis_df('df_spy')
df_vix = get_redis_df('df_vix')
df_tnx = get_redis_df('df_tnx')
df_1yr_rate = get_redis_df('df_1yr_rate')
df_div = get_redis_df('df_div')


In [5]:
df_spy.tail()

,high,low,open,close,volume,date,settle_date
7553,3111.510010,3047.830078,3050.199951,3100.290039,4696280000,2020-06-30,20200630
7554,3128.439941,3101.169922,3105.919922,3115.860107,4443130000,2020-07-01,20200701
7555,3165.810059,3124.520020,3143.639893,3130.010010,4190830000,2020-07-02,20200702
7556,3182.590088,3155.290039,3155.290039,3179.719971,4736450000,2020-07-06,20200706
7557,3184.149902,3159.290039,3166.439941,3170.360107,1248297311,2020-07-07,20200707


In [ ]:
df_cuv_implied = get_redis_df('df_cuv_implied')

In [ ]:
df_cuv_implied[df_cuv_implied.symbol.str.slice(0,2)=='CL']